# ДЗ №6 - Анализ настроя отзыва с использованием LSTM

В этой работе предлагается реализовать рекуррентную нейронную сеть, которая выполняет задачу определения настроя отзыва на фильм.

Здесь использвается набор данных отзывов на фильмы, каждый из которых помечен как положительный или отрицательный.

<img src="assets/reviews_ex.png" width=40%>

### Архитектура нейросети

Ниже показана архитектура нейросети, которую предлагается реализовать.

<img src="assets/network_diagram.png" width=40%>

**Первым делом каждое слово отправляется на вход слоя извлечения скрытого представления (эмбеддингов)** Эмбеддинги нужны для того, чтобы десятки тысяч слов, которые потенциально могут быть использованы в отзывах, представить в более эффективном виде, нежели one-hot представление. Вы знакомы с понятием эмбеддингов, например, из лекции про Word2Vec. Эмбеддинги можно тренировать предварительно в подходе Word2Vec или использовать любую другую модель извлечения эмбеддингов. Однако в этой задаче достаточно вставить слой преобразования слов в эмбеддинги, чтобы на основании предоставленных на обучение данных он сам выучил наиболее эффективное скрытое представлени слов. Здесь слой извлечение эмбеддингов - скорее для снижения размерности, нежели для извлечения значимого семантического представления.

**После того, как отдельные слова переданы в слой извлечения эмбеддингов, эти скрытые представления слов подаются на ячейку LSTM.** Ячейки LSTM - элементы рекуррентной нейронной сети, которые дают сети возможность усваивать информацию о последовательности слов в отзыве.

**Выходным слоем после LSTM будет сигмоидальная активация.** Поскольку в случае двух взаимоисключающих исходов для каждого объекта (позитивный и негативный настрой отзыва) - можно решать задачу бинарной классификации. Для такой задачи финальная активация - сигмоид.

Результат выдается ячейкой LSTM на каждом слове, однако нас интересует только финальный ответ - то есть, активация на последнем слове. Все предыдущие можно игнорировать. Соответственно, функция потерь (бинарная перекрестная энтропия) вычисляется для всего высказывания (отзыва) в целом, на значении активации на последнем слове.

---
### Загрузим и отобразим данные

In [1]:
import numpy as np

# read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [3]:
print(reviews[:1000])
print()
print(labels[:100])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

## Предобработка данных

Первым делом в решении подобных задач следует преобразовать данные к виду, который можно подавать на вход нейросети. Мы решили использовать обучаемый слой извлечения эмбеддингов, поэтому каждое слово следует закодировать. В этой работе предлагается закодировать слово просто номером в словаре. Для составления словаря, данные следует почистить:

* Удалить знаки препинания
* Привести все слова в нижний регистр
* Отзывы разделены знаком переноса строки `\n`. Следует разбить весь текст на отдельные отзывы по этому символу (отдельные отзывы понадобятся позже)
* Объединить все отзывы без лишних символо в один длинный текст

In [5]:
from string import punctuation

print(punctuation)

# Удалить знаки препинания
reviews = reviews.lower() # перевести в нижний регистр
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [7]:
# разделить на отдельные отзывы
reviews_split = all_text.split('\n')

# объединить в один большой текст
all_text = ' '.join(reviews_split)

In [8]:
# Создать список всех слов
words = all_text.split()

In [9]:
words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

### Кодирование слов

Таблица соответствий (lookup-table) должна давать возможность идентифицировать каждое слово слову целым числом. Самый простой способ это сделать - создать словарь, ключами которого будут слова, значениями - целые числа. С использованием такой таблицы соответствий можно будет преобразовать все объекты выборки (отзывы на фильмы) в последовательность целых чисел.

> **ЗАДАНИЕ 1:** Здесь следует закодировать слова целыми числами. Для упрощения извлечения уникальных слов и частоты их повторяемости можно использовать класс `Counter` модуля `collections` и его возможности сортировки по частоте элементов. Обратите внимание, что в дальнейшем потребуется использовать специальный символ отступа в последовательности, который будет кодироваться нулем `0`, поэтому при составлении таблицы соответствий **следует начинать нумерацию с `1`, а не с `0`**.
> Далее преобразуйте все обзоры, записанные в переменной reviews, согласно сформированным целочисленным кодам. Запишите сконвертированные обзоры в новый список `reviews_ints`.

In [ ]:
# feel free to use this import 
from collections import Counter

vocab_to_int = None

reviews_ints = []



**Протестируйте свой код**

Проверьте количество уникальных слов в полученном словаре. Должно получиться более 74'000

Также можно посмотреть, как выглядит произвольный из токенизированных обзоров.

In [ ]:
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

print('Tokenized review: \n', reviews_ints[:1])

### Кодирование целевой переменной

Метки обзоров имеют два возможных значения: "positive" или "negative". Чтобы использовать эти метки, их следует преобразовать к виду бинарной переменной.

> **ЗАДАНИЕ 2:** Преобразуйте данные целевой переменной из слов `positive` и `negative` в бинарное значени (`1` и `0` соответственно). Запишите данные в массив целевой переменной  `encoded_labels`.

In [ ]:
# Преобразование меток отзывов: 1=positive, 0=negative
encoded_labels = None

### Фильтрация выбросов

Длина обзоров, подаваемых на вход нейросети, должна будет быть одинаковая. Недостаток слов легко заполняется токеном-пропуском. Однако если отзыв необычно длинный, имеет смысл исключить его из рассмотрения. Это можно сделать следующим образом:

1. Исключить экстремально длинные и экстремально короткие отзывы
2. Заполнение недостающих (в смысле длины отзыва) слов специальным токеном-пропуском, чтобы все отзывы были одинаковые по длине.
3. Уменьшение длины отзывов, превышающих установленный (вами) размер.

<img src="assets/outliers_padding_ex.png" width=40%>

Первым делом следует исключить из выборки экстремально длинные и экстремально короткие отзывы. Как минимум следует исключить отзывы с нулевой длиной.

>**ЗАДАНИЕ 3:** Следует отфильтровать отзывы по длине. Например, можно удалить из `reviews_ints` отзывы длиной короче перцентиля уровня 1% и длинее перцентиля уровня 99%. Запишите отфильтрованные таким образом отзывы снова в `reviews_ints`, а соответствующие им метки - в список `encoded_labels`

In [ ]:
# outlier review stats
review_lengths = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))


In [ ]:
print('Number of reviews before removing outliers: ', len(reviews_ints))


reviews_ints = # YOUR CODE HERE
encoded_labels = # YOUR CODE HERE

print('Number of reviews after removing outliers: ', len(reviews_ints))

---
## Заполнение недостающих токенов (padding)

Для того, чтобы обрабатывать отзывы, превышающие задаваемую (вами) длину или не заполняющую ее целиком, следует применить заполнение недостающих токеном (т.н. padding) и отбросить лишние токены отзыва. Таким образом, все отзывы будут одной длины. Для отзывов, которые короче, чем некоторый `seq_length`, слева следует добавить ровно столько токенов пустого слова (то есть, `0`), чтобы их длина стала равной `seq_length` ("padding"). Для слишком длинный отзывов можно отбросить последние токены таким образом, чтобы их длина также сравнялась с `seq_length` ("truncate"). Предлагаем использовать `seq_length`, равный 200. Однако этот параметр можно попробовать варьировать.

> **ЗАДАНИЕ 4:** Реализовать преобразование (padding + truncate) в виде фунации, которая бы возвращала массив целочисленных признаков слов отзывов (названы `features` в заготовке кода ниже). При этом все отзывы по результату выполнения функции должны становиться одинаковой длины `seq_length`.

* Данные отзывов берутся из набора закодированных отзывов `review_ints`;
* Каждый массив закодированого отзыва должен получиться длины `seq_length`;
* Для отзывов, которые короче `seq_length`, применяется левое дополнение ("left pad") пустым токеном `0`. То есть, закодированный отзыв длиной 3 вида `[117, 18, 128]` должен быть преобразован к виду `[0, 0, 0, ..., 0, 117, 18, 128]`;
* Для отзывов, которые длинее `seq_length`, отбрасываются правые (последние) токены таким образом, чтобы результирующая длина составляла `seq_length`.

**ПРИМЕР**

Пусть `seq_length=10`. Тогда для отзыва, закодированного в виде: 
```
[117, 18, 128]
```
Результат должен выглядеть следующим образом: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

В конечном итоге, массив признакового описания отзывов `features` должен быть 2D массивом. Количество строк должно совпадать с количеством отзывов в `review_ints`. Количество колонок должно быть `seq_length`.

In [ ]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## YOUR CODE HERE
    
    features=None
    
    return features

In [ ]:
# Протестируйте ваш код

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## здесь тестируется результат - НЕ МЕНЯЙТЕ ЭТОТ КОД - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# первые 10 значений первых 30 мини-батчей
print(features[:30,:10])
## здесь тестируется результат - НЕ МЕНЯЙТЕ ЭТОТ КОД - ##

## Разбиение на выборки: тренировочную, валидационную и тестовую.

Теперь пора приступить к обычной процедуре разбиения доступной выборки данных на подвыборки для тренировки, валидации и тестирования.

> **ЗАДАНИЕ 5:** разбить данные на три выборки: тренировочная, валидационная и тестовая. 
* Следует создать тренировочную подвыборку объемом `split_frac` от всей доступной.
* оставшиеся данные разбейте пополам на валидационную и тестовую.

In [ ]:
split_frac = 0.8

## Разбейте данные на выборки: тренировочную, валидационную и тестовую


## Покажите размеры полученных выборок


---
## Загрузчик данных (DataLoader) и разбиение на мини-батчи

Далее следует создать загрузчик данных для каждой из подвыборок. Для этого можно воспользоваться [TensorDataset](https://pytorch.org/docs/stable/data.html#) из массивов `numpy`, которые были созданы. Далее можно воспользоваться классом DataLoader, чтобы разбить полученные наборы данных на мини-батчи и организовать порождение мини-батчей.


Например, это может выглядеть так:
```
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)
```

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# не забудьте перемешать данные!
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
# Отобразите один элемент из потока порождаемых мини-батчей
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

---
# Искусственная нейронная сеть, определяющая настрой отзыва

Теперь настало время реализовать нейросеть, которая будет собственно выполнять задачу.

<img src="assets/network_diagram.png" width=40%>

Предлагается реализовать нейросеть в следующем составе слоев:
1. Слой извлечения эмбеддингов ([embedding layer](https://pytorch.org/docs/stable/nn.html#embedding)). Этот слой преобразует токены слов (целые числа) в эмбединги задаваемого размера;
2. [Слой LSTM](https://pytorch.org/docs/stable/nn.html#lstm). Для этого слоя следует задать размер скрытого состояния `hidden_state` и количество слоев;
3. Полносвязный слой на выходе LSTM, который преобразует выход LSTM в переменную задаваемого размера `output_size` (напомним, в этой задаче целевая переменная - только одно значение).
4. Сигмоидальная активация. Напомним, что нейросеть должна возвращать вывод **только последнего шага рекуррентной ячейки**.

### Слой извлечения эмбеддингов

В словаре, сформированном в этой задаче, - более 74000 слов. Для сокращения размерности и повышения эффективности вычислений следует применять слой извлечения эмбеддингов слов, [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding). Можно было бы тренировать этот слой методом, аналогичным Word2Vec. Однако конкретно в этой задаче можно тренировать этот слой одновременно со всей нейросетью, в подходе end-to-end.


### Слои LSTM

Чтобы нейросеть была собственно рекуррентной, можно, например, использовать слои [LSTM](https://pytorch.org/docs/stable/nn.html#lstm). Изучите по документации назначение параметров этой ячейки! Можно экспериментировать с количеством слоев ячейки. Скорее всего, при повышении количества слоев `n_layers` ячейка будет способна усваивать и описывать более сложные закономерности в данных.

> **ЗАДАНИЕ 6:** Реализуйте методы `__init__`, `forward`, и `init_hidden` в классе модели SentimentRNN.

Заметим, что в методе `init_hidden` все скрытые состояния и состояния ячеек LSTM следует обнулить. Если есть возможность, эти состояния следует отправить на GPU в этом же методе.

In [ ]:
# Проверка, доступен ли GPU
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    Класс модели рекуррентной сети, предназначенной для выполнения бинарной классификации текстов
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Конфигурация модели: создание объектов слоев сети
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        

    def forward(self, x, hidden):
        """
        Вычисление сети на некоторых объектах выборки и скрытых состоянии
        """
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Инициализация скрытых состояний '''
        # Создать два новых тензора размеров [n_layers, batch_size, hidden_dim]
        # для скрытых состояний и состояний ячейки. LSTM
        # Обнулить созданные тензоры и (при необходимости) отрпавить их на GPU
        
        return hidden
        

## Создание экземпляра класса нейросети

> **ЗАДАНИЕ 7:** Следует создать экземпляр класса описанной выше нейросети с заданием следующих гиперпараметров:

* `vocab_size`: Размер словаря (для слоя эмбеддингов)
* `output_size`: Размер вектора результата на нейросети (1 в случае бинарной классификации)
* `embedding_dim`: Количество признаков векторов эмбеддингов
* `hidden_dim`: Количество "нейронов" скрытого слоя ячейки LSTM. Обычно чем больше, - тем лучше. Значения, которые можно рассматривать как первое приближение: 128, 256, 512, *etc.*
* `n_layers`: Количество слоев LSTM. Неплохим первым приближением может быть количество 1-3

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = 
output_size = 
embedding_dim = 
hidden_dim = 
n_layers = 

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

---
## Обучение нейросети

Здесь код не будет отличаться от того, к чему вы уже привыкли. Здесь следует реализовать стандартный цикл обучения.

> **ЗАДАНИЕ 8:** реализовать цикл обучения нейросети с периодической проверкой качества на валидационной выбрке.

In [ ]:
# Функция потерь и оптимизатор
criterion = None
optimizer = None

In [ ]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        
        output, h = net(inputs, h)
        
        # YOUR CODE HERE

        # loss stats
        if counter % print_every == 0:
            
            
            
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Batch: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

---
## Тестирование нейросети

Тестировать обученную нейросеть можно несколькими способами.

* **В мере качества, оцениваемой на тестовой выборке**

* **Оценка правдоподобности результата на отзывах, написанных исследователем.**

> **ЗАДАНИЕ 9:** оцените качество модели на тестовой выборке.

In [ ]:
test_losses = []
num_correct = 0

h = net.init_hidden(batch_size)

net.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = None
    test_losses.append(test_loss.item())
    
    # output -> label
    pred = None
    
    # Вычисление Accuracy
    

# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

### Применение модели на тестовом отзыве

> **ЗАДАНИЕ 10:** реализовать фунацию, `predict`, которая принимает на вход обученную сеть, текст отзыва, длину последовательности `seq_length`, кодирует текст в последовательность целочисленных кодов, применяет  padding и truncate и выводит результат, является ли настрой отзыва положительным или отрицательным.

In [10]:
# отрицательный отзыв
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'


In [ ]:
def predict(net, test_review, sequence_length=200):
    ## YOUR CODE HERE
    
        

In [ ]:
# Положительный отзыв
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [ ]:
# Примените фунацию. Какой ответ выдает нейросеть?
# Попробуйте оба отзыва. Попробуйте отызвы, которые сможете найти в Интернете.
seq_length=200
predict(net, test_review_neg, seq_length)